##### Title: Monitoring how RTT varies based on the geolocation of the target
##### Author: Lotito Davide Pio
##### Date: 
<img src="./images/logoUnipvST.jpg" width="75" height="75">

Some useful imports

In [5]:

from matplotlib import pyplot as plt
import pandas as pd

Read the file 

In [8]:
df = pd.read_json("../sample.txt")
df

,destination,date,time,packetLost,excutionTime,minRTT,avgRTT,maxRTT,devRTT
0,142.251.209.14,2022-04-10,08:28:28,0%,9012ms,12.323,15.037,23.502,3.502
1,31.13.86.36,2022-04-10,08:28:28,0%,9012ms,11.742,14.680,21.308,3.298
2,140.82.121.4,2022-04-10,08:28:28,0%,9014ms,20.891,22.026,24.049,0.977
3,13.107.42.14,2022-04-10,08:28:28,0%,9013ms,12.079,14.373,24.648,3.552
